# Create and query knowledge graph with LLM. (neo4j version)
![title](neo4jdogs.png)

We created a pickle file in notebook ´[Create knowledge graph from PDF](./knowledgegraph.ipynb)´ that we will now use with Neo4J

In [ ]:
%pip install -r requirements.txt

# Load env variables and connect to neo4j database
Please run _docker-compose up_ first on the directory to start the database.    

In [ ]:
import os
from langchain.graphs import Neo4jGraph
from dotenv import load_dotenv

load_dotenv()

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)


## Load documents to Neo4J.

In [ ]:
# Only execute this cell if you need to load the graph from pickle (like after kernel restart)
import pickle
with open('./data/graph_docs.pkl','rb') as f:
    graph_docs = pickle.load(f)
    graph.add_graph_documents(graph_docs)
    

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import AzureChatOpenAI

graph.refresh_schema()

# Need new llm, cypher chain does not work with json output
c_llm = AzureChatOpenAI(
    model=os.getenv("OPENAI_DEPLOYMENT_NAME"), 
    temperature=0, 
    max_tokens=1500,
    verbose=True)

cypher_chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=c_llm,
    qa_llm=c_llm,
    validate_cypher=True,
    verbose=True
)

In [ ]:
from IPython.display import Markdown, display
response = cypher_chain.invoke("Make table of different breeding groups and their characteristics")
display(Markdown(response["result"]))

In [ ]:
# This empties the Neo4j database
graph.query("MATCH (n) DETACH DELETE n")